# Testing

In [4]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from pyexplainer.pyexplainer_pyexplainer import PyExplainer
import pickle
import os


cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
path_train = parent_dir + "/pyExplainer/tests/pyexplainer_test_data/activemq-5.0.0.zip"
training_data = pd.read_csv(path_train, index_col = 'File')

dep = training_data.columns[-4]
selected_features = ["ADEV", "AvgCyclomaticModified", "AvgEssential", "AvgLineBlank", "AvgLineComment",
                     "CountClassBase", "CountClassCoupled", "CountClassDerived", "CountDeclClass",
                     "CountDeclClassMethod", "CountDeclClassVariable", "CountDeclInstanceVariable",
                     "CountDeclMethodDefault", "CountDeclMethodPrivate", "CountDeclMethodProtected",
                     "CountDeclMethodPublic", "CountInput_Mean", "CountInput_Min", "CountOutput_Min", "MAJOR_LINE",
                     "MaxInheritanceTree", "MaxNesting_Min", "MINOR_COMMIT", "OWN_COMMIT", "OWN_LINE",
                     "PercentLackOfCohesion", "RatioCommentToCode"]
all_cols = training_data.columns
for col in all_cols:
    if col not in selected_features:
        all_cols = all_cols.drop(col)
indep = all_cols
X_train = training_data.loc[:, indep]
y_train = training_data.loc[:, dep]

blackbox_model = RandomForestClassifier(max_depth=3, random_state=0)
blackbox_model.fit(X_train, y_train)

class_label = ['Clean', 'Defect']
pyExp = PyExplainer(X_train,
            y_train,
            indep,
            dep,
            blackbox_model,
            class_label=class_label)

path_test = parent_dir + "/pyExplainer/tests/pyexplainer_test_data/activemq-5.1.0.zip"
sample_test_data = pd.read_csv(path_test, index_col = 'File')
X_test = sample_test_data.loc[:, indep]
y_test = sample_test_data.loc[:, dep]

sample_explain_index = 0
pyExp.X_explain = X_test.iloc[[sample_explain_index]]
pyExp.y_explain = y_test.iloc[[sample_explain_index]]

# Util functions for reading and writing data
def save_object(object_i, filename):
    with open(filename, 'wb') as file:
        pickle.dump(object_i, file)

def load_object(filename):
    with open(filename, 'rb') as file:
        object_o = pickle.load(file)
    return (object_o)

path_rule = parent_dir + "/pyExplainer/tests/rule_objects/rule_object.pyobject"
# load rule obj
if os.path.isfile(path_rule):
    load_pyExp_rule_obj = load_object(path_rule)

In [2]:
%%time

d = []
for i in range(1000):
    X_explain = X_test.iloc[[i]]
    y_explain = y_test.iloc[[i]]
    pyExp.X_explain = X_test.iloc[[i]]
    pyExp.y_explain = y_test.iloc[[i]]
    rule = pyExp.explain(X_explain, y_explain, max_iter=10000)
    top_rule = pyExp.parse_top_rules(top_k_positive_rules=rule['top_k_positive_rules'],
                                     top_k_negative_rules=rule['top_k_negative_rules'])
    d.append(pyExp.generate_bullet_data(top_rule))


KeyboardInterrupt: 

In [6]:
i = 5
X_explain = X_test.iloc[[i]]
y_explain = y_test.iloc[[i]]
pyExp.X_explain = X_test.iloc[[i]]
pyExp.y_explain = y_test.iloc[[i]]
rule = pyExp.explain(X_explain, y_explain, max_iter=10000)
pyExp.visualise(rule)

FloatSlider(value=76.0, continuous_update=False, description='#1 Decrease the values of PercentLackOfCohesion …

FloatSlider(value=2.0, continuous_update=False, description='#2 Increase the values of AvgCyclomaticModified t…

FloatSlider(value=0.88, continuous_update=False, description='#3 Increase the values of RatioCommentToCode to …

Output(layout=Layout(border='3px solid black'), outputs=({'output_type': 'display_data', 'data': {'text/plain'…

In [3]:
pyExp.bullet_data

[{'title': '#1 Decrease the values of PercentLackOfCohesion to less than 0',
  'subtitle': 'Actual = 0',
  'ticks': [0, 87],
  'step': [1],
  'startPoints': [0, 398.0],
  'widths': [398.0, 52.0],
  'colors': ['#a6d96a', '#d7191c'],
  'markers': [46.0],
  'varRef': 'PercentLackOfCohesion'},
 {'title': '#2 Decrease the values of CountDeclMethodPublic to less than 1',
  'subtitle': 'Actual = 1',
  'ticks': [1, 32],
  'step': [1],
  'startPoints': [0, 262.0],
  'widths': [262.0, 188.0],
  'colors': ['#a6d96a', '#d7191c'],
  'markers': [1],
  'varRef': 'CountDeclMethodPublic'},
 {'title': '#3 Increase the values of RatioCommentToCode to more than 1.54',
  'subtitle': 'Actual = 1.54',
  'ticks': [0, 2],
  'step': [0.1],
  'startPoints': [0, 70.0],
  'widths': [70.0, 380.0],
  'colors': ['#d7191c', '#a6d96a'],
  'markers': [1.54],
  'varRef': 'RatioCommentToCode'}]